In [2]:
import utils
utils.rcparams()

In [5]:
herschel = utils.full_loader('herschel')
vla = utils.full_loader('VLA')

In [ ]:
class Object:

    def __init__(self, ra, dec):
        self.ra = ra
        self.dec = dec


In [ ]:
def create 

In [ ]:
class Survey:

    def __init__(self):
